# Load Environments

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv('openApiKey')
base_url = os.getenv('openApiBaseUrl')

# Import Libraries 

In [23]:
from langchain_openai import ChatOpenAI , OpenAIEmbeddings
from langchain_chroma import Chroma
from pydantic import BaseModel, Field
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage , AIMessage , SystemMessage
from langchain_core.prompts import ChatPromptTemplate , PromptTemplate , MessagesPlaceholder
from langchain_core.documents import Document
from langchain_core.runnables import chain , RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_classic.output_parsers import PydanticOutputParser
from langchain_classic.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

# Intialize the Model 

In [24]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=api_key, openai_api_base=base_url, temperature=0)

# Intialize Structured Output

In [27]:
class CountryCapital(BaseModel):
    country: str = Field(description="The name of the country")
    capital: str = Field(description="The capital city of the country")


In [28]:
parser = PydanticOutputParser(pydantic_object=CountryCapital)

prompt = PromptTemplate(
    template="Provide the capital of {country}. {format_instructions}",
    input_variables=["country"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [ ]:
chain = prompt | llm | parser

In [8]:
response = chain.invoke("Japan")
print(response)

country='Japan' capital='Tokyo'


# Intialize Memory

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

In [36]:
memory = ConversationBufferMemory(return_messages=True)

In [35]:
chain = prompt | llm | StrOutputParser()

In [37]:
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [38]:
conversation = RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=get_session_history,
    input_messages_key="input",    
    history_messages_key="history", 
)

session_id = "user1"

In [39]:
response1 = conversation.invoke(
    {"input": "Hello, who are you?"},
    config={"configurable": {"session_id": session_id}},
)
print("\nAssistant:", response1)


Assistant: Hello! I am a helpful assistant here to provide you with information and assistance. How can I help you today?


In [40]:
response2 = conversation.invoke(
    {"input": "Can you remind me what I said earlier?"},
    config={"configurable": {"session_id": session_id}},
)
print("\nAssistant:", response2)


Assistant: Of course! You asked me who I am. How can I assist you further?


# Structured Output

1️⃣ StructuredOutput

`Concept:` StructuredOutput is a base class / framework in LangChain that defines how outputs from LLMs should be structured.

- Instead of just returning plain text, we want the output in a specific format, e.g., JSON, dictionary, or a Pydantic model.

`This is useful when:`

- You want consistent parsing for APIs.

- You want to extract multiple fields (e.g., name, age, country) reliably.

In [2]:
from langchain_core.output_parsers import StrOutputParser

class PersonOutput(StrOutputParser):
    name: str
    age: int

2️⃣ StructuredOutputParser with Pydantic

`Purpose:` Wraps a Pydantic model to parse LLM outputs into structured Python objects.

`Pydantic ensures:`

- Type validation (int, str, float, etc.)

- Required fields are present

- Nested structures work

In [4]:
from pydantic import BaseModel
from langchain_core.output_parsers import StrOutputParser

class PersonModel(BaseModel):
    name: str
    age: int

parser = StrOutputParser(pydantic_object=PersonModel)
text = '{"name": "Alice", "age": 25}'
parsed = parser.parse(text)

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
output = parser.parse("  Hello world!  ")
print(output)  # "Hello world!"

  Hello world!  


5️⃣ PydanticOutputParser

Shortcut for StructuredOutputParser with Pydantic .
Takes a Pydantic model and ensures LLM output matches that model.

`Automatically:`

- Validates fields

- Converts types

- Throws descriptive errors if invalid

In [6]:
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=PersonModel)
parsed = parser.parse('{"name": "Bob", "age": 30}')
print(parsed.name)

Bob


# tools

In [14]:
from langchain_core.tools import tool ,StructuredTool

In [9]:
@tool 
def mul (a:int ,b:int) -> int : 
    """Multiply 2 numbers"""
    return a*b

In [10]:
mul.invoke({"a":3 , "b":6})

18

In [13]:
mul.name , mul.description , mul.args , mul.args_schema.model_json_schema()

('mul',
 'Multiply 2 numbers',
 {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 {'description': 'Multiply 2 numbers',
  'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'title': 'B', 'type': 'integer'}},
  'required': ['a', 'b'],
  'title': 'mul',
  'type': 'object'})